In [1]:
from greensight.processing import load_sentinel_data_from_dir
from greensight.utils import DIR_DATA
import pandas as pd

In [2]:
sentinel_data_path = DIR_DATA / "sentinel_data"
assert sentinel_data_path.is_dir()

In [ ]:
# load data for each year
condition = lambda path: "mean" in path.stem and "S1" in path.stem

dfs = []
for path in sentinel_data_path.iterdir(): 
    if "2016" not in path.name:
        print("pass: ", path)
        df = load_sentinel_data_from_dir(path, condition=condition)
        print(df.shape)
        dfs.append(df)
        print()
    else:
        print("fail: ", path)

fail:  /home/finley/Work/RDS/greensight/data/sentinel_data/Greenbelts_2016
pass:  /home/finley/Work/RDS/greensight/data/sentinel_data/Greenbelts_2017


/home/finley/Work/RDS/greensight/greensight/processing.py:27: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  band_inds = pd.unique(
/home/finley/Work/RDS/greensight/greensight/processing.py:32: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  month_inds = pd.unique(
/home/finley/Work/RDS/greensight/greensight/processing.py:73: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_month = df_month.groupby(level=[0, 1], axis=1).mean()


(12, 720)

pass:  /home/finley/Work/RDS/greensight/data/sentinel_data/Greenbelts_2018


In [14]:
path.name

'Greenbelts_2017'

In [4]:
# combine years
df_comb = pd.concat(dfs, axis=0)
df_comb.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " ").replace(",", ""), i[1], i[2]) for i in df_comb.columns])
df_comb.columns.names =  ("greenbelt", "shape", "band")
df_comb.head(5)

greenbelt  Merseyside and Greater Manchester                                   \
shape                              E06000006                                    
band                                   VHAsc      VHDes      VVAsc      VVDes   
2016-01-01                        -21.880295 -16.991904 -15.775903  -9.790099   
2016-02-01                        -23.155242 -17.578793 -16.930790  -9.725071   
2016-03-01                        -23.382046 -18.103710 -17.087475 -10.531700   
2016-04-01                        -24.581625 -17.473175 -19.243807 -10.768544   
2016-05-01                        -23.044508 -18.529830 -18.120999 -12.751782   

greenbelt                                                                     \
shape       E06000007                                   E06000008              
band            VHAsc      VHDes      VVAsc      VVDes      VHAsc      VHDes   
2016-01-01 -16.071707 -16.330632  -9.220326  -9.751699 -16.652340 -16.737056   
2016-02-01 -16.492496 -16.878869  -9.339440  -9.714068 -16.769975 -16.911006   
2016-03-01 -16.863572 -17.473715  -9.546583 -10.452768 -17.058511 -18.111202   
2016-04-01 -16.929850 -16.801029 -10.216895 -10.500869 -17.323229 -17.503710   
2016-05-01 -17.919154 -17.971877 -12.141177 -12.285496 -18.045142 -18.141680   

greenbelt   ...    London                                              \
shape       ... E09000027             E09000029                         
band        ...     VVAsc      VVDes      VHAsc      VHDes      VVAsc   
2016-01-01  ... -8.756936 -10.676782 -16.771370 -17.255673 -10.597951   
2016-02-01  ... -8.943721 -11.705665 -17.185180 -17.619796 -10.872856   
2016-03-01  ... -9.011251 -11.404164 -17.095400 -17.859541 -10.820224   
2016-04-01  ... -9.391430 -11.195927 -17.181280 -17.307712 -11.139923   
2016-05-01  ... -9.288337 -11.733921 -17.733808 -18.182160 -11.964601   

greenbelt                                                          
shape                  E09000031                                   
band            VVDes      VHAsc      VHDes      VVAsc      VVDes  
2016-01-01 -11.723700 -16.742193 -17.174760 -10.834776 -11.552687  
2016-02-01 -11.681797 -16.307043 -17.452870 -10.085572 -11.418674  
2016-03-01 -12.022884 -16.547483 -17.510375 -10.391498 -11.840899  
2016-04-01 -11.771096 -16.631947 -17.335087 -10.686377 -11.707105  
2016-05-01 -12.637640 -17.542597 -18.446995 -11.333272 -12.681095  

[5 rows x 720 columns]

In [5]:
# save DataFrame
output_path = DIR_DATA / "processed_data/sentinel_one_df.hdf"
df_comb.to_hdf(output_path, key="df", mode="w", format="table")

In [6]:

## check it is able to be loaded
df_loaded = pd.read_hdf(output_path, key="df")
df_loaded.head(5)

greenbelt  Merseyside and Greater Manchester                                   \
shape                              E06000006                                    
band                                   VHAsc      VHDes      VVAsc      VVDes   
2016-01-01                        -21.880295 -16.991904 -15.775903  -9.790099   
2016-02-01                        -23.155242 -17.578793 -16.930790  -9.725071   
2016-03-01                        -23.382046 -18.103710 -17.087475 -10.531700   
2016-04-01                        -24.581625 -17.473175 -19.243807 -10.768544   
2016-05-01                        -23.044508 -18.529830 -18.120999 -12.751782   

greenbelt                                                                     \
shape       E06000007                                   E06000008              
band            VHAsc      VHDes      VVAsc      VVDes      VHAsc      VHDes   
2016-01-01 -16.071707 -16.330632  -9.220326  -9.751699 -16.652340 -16.737056   
2016-02-01 -16.492496 -16.878869  -9.339440  -9.714068 -16.769975 -16.911006   
2016-03-01 -16.863572 -17.473715  -9.546583 -10.452768 -17.058511 -18.111202   
2016-04-01 -16.929850 -16.801029 -10.216895 -10.500869 -17.323229 -17.503710   
2016-05-01 -17.919154 -17.971877 -12.141177 -12.285496 -18.045142 -18.141680   

greenbelt   ...    London                                              \
shape       ... E09000027             E09000029                         
band        ...     VVAsc      VVDes      VHAsc      VHDes      VVAsc   
2016-01-01  ... -8.756936 -10.676782 -16.771370 -17.255673 -10.597951   
2016-02-01  ... -8.943721 -11.705665 -17.185180 -17.619796 -10.872856   
2016-03-01  ... -9.011251 -11.404164 -17.095400 -17.859541 -10.820224   
2016-04-01  ... -9.391430 -11.195927 -17.181280 -17.307712 -11.139923   
2016-05-01  ... -9.288337 -11.733921 -17.733808 -18.182160 -11.964601   

greenbelt                                                          
shape                  E09000031                                   
band            VVDes      VHAsc      VHDes      VVAsc      VVDes  
2016-01-01 -11.723700 -16.742193 -17.174760 -10.834776 -11.552687  
2016-02-01 -11.681797 -16.307043 -17.452870 -10.085572 -11.418674  
2016-03-01 -12.022884 -16.547483 -17.510375 -10.391498 -11.840899  
2016-04-01 -11.771096 -16.631947 -17.335087 -10.686377 -11.707105  
2016-05-01 -12.637640 -17.542597 -18.446995 -11.333272 -12.681095  

[5 rows x 720 columns]